## 🌀 unravel kloppy into graph neural network using the _new_ Polars back-end!

### ‼️ NEW PYTORCH VERSION

First run `pip install unravelsports` if you haven't already!


-----


In [1]:
# %pip install unravelsports torch torch-geometric pytorch-lightning --quiet

In this in-depth walkthrough we'll discuss everything the `unravelsports` package has to offer for converting a [Kloppy](https://github.com/PySport/kloppy) dataset of soccer tracking data into graphs for training binary classification graph neural networks using PyTorch Geometric and PyTorch Lightning, with a newly added (version==0.3.0+) [Polars](https://pola.rs/) back-end.

This walkthrough will touch on a lot of the concepts from [A Graph Neural Network Deep-dive into Successful Counterattacks {A. Sahasrabudhe & J. Bekkers}](https://github.com/USSoccerFederation/ussf_ssac_23_soccer_gnn). It is strongly advised to first read the [research paper (pdf)](https://ussf-ssac-23-soccer-gnn.s3.us-east-2.amazonaws.com/public/Sahasrabudhe_Bekkers_SSAC23.pdf). Some concepts are also explained in the [Graphs FAQ](graphs_faq.md).

Step by step we'll show how this package can be used to load soccer positional (tracking) data with `kloppy`, how to convert this data into a `KloppyPolarsDataset`, convert it into "graphs", train a Graph Neural Network with PyTorch Geometric, evaluate its performance, save and load the model and finally apply the model to unseen data to make predictions.

The powerful Kloppy package allows us to load and standardize data from many providers: Metrica, Sportec, Tracab, SecondSpectrum, StatsPerform and SkillCorner. In this guide we'll use some matches from the [Public Sportec (DFL) Dataset (Bassek et al. 2025)](https://www.nature.com/articles/s41597-025-04505-y).

<br>
<i>Before we get started it is important to note that the <b>unravelsports</b> library does not have built in functionality to create binary labels, these will need to be supplied by the reader. In this example we use dummy labels instead. 
</i>
<br>

##### **Contents**

- [**1. Imports**](#1-imports).
- [**2. Public Sportec (DFL) Data**](#2-public-sportec-data).
- [**3. ⭐ _KloppyPolarsDataset_ and _SoccerGraphConverter_**](#3-kloppypolarsdataset-and-soccergraphconverter).
- [**4. Load Kloppy Data, Convert & Store**](#4-load-kloppy-data-convert-and-store).
- [**5. Creating a Custom Graph Dataset**](#5-creating-a-custom-graph-dataset).
- [**6. Prepare for Training**](#6-prepare-for-training).
    - [6.1 Split Dataset](#61-split-dataset)
    - [6.2 Model Configurations](#62-model-configurations)
    - [6.3 Build GNN Model](#63-build-gnn-model)
    - [6.4 Create DataLoaders](#64-create-dataloaders)
- [**7. GNN Training + Prediction**](#7-training-and-prediction).
    - [7.1 Initialize Trainer](#71-initialize-trainer)
    - [7.2 Train Model](#72-train-model)
    - [7.3 Save & Load Model](#73-save--load-model)
    - [7.4 Evaluate Model](#74-evaluate-model)
    - [7.5 Predict on New Data](#75-predict-on-new-data)

ℹ️ [**Graphs FAQ**](graphs_faq.md)

-----

### 1. Imports

We import `SoccerGraphConverter` to help us convert from Kloppy positional tracking frames to graphs.

With the power of **Kloppy** we can also load data from many providers by importing `metrica`, `sportec`, `tracab`, `secondspectrum`, `signality`, `pff`, `hawkeye` or `statsperform` from `kloppy`.

In [2]:
from unravel.soccer import SoccerGraphConverter, KloppyPolarsDataset

from kloppy import sportec

-----

### 2. Public Sportec Data

The `SoccerGraphConverter` class allows processing data from every tracking data provider supported by [PySports Kloppy](https://github.com/PySport/kloppy), namely:
- Sportec
- Tracab
- SecondSpectrum
- SkillCorner
- StatsPerform
- Metrica
- PFF (beta)
- HawkEye (alpha)
- Signality (alpha)

You can choose any of the following games from the open Sportec dataset:

```python
matches = {
    'J03WMX': "1. FC Köln vs. FC Bayern München",
    'J03WN1': "VfL Bochum 1848 vs. Bayer 04 Leverkusen",
    'J03WPY': "Fortuna Düsseldorf vs. 1. FC Nürnberg",
    'J03WOH': "Fortuna Düsseldorf vs. SSV Jahn Regensburg",
    'J03WQQ': "Fortuna Düsseldorf vs. FC St. Pauli",
    'J03WOY': "Fortuna Düsseldorf vs. F.C. Hansa Rostock",
    'J03WR9': "Fortuna Düsseldorf vs. 1. FC Kaiserslautern"
}
```

-----

### 3. ⭐ _KloppyPolarsDataset_ and _SoccerGraphConverter_

ℹ️ For more information on:
- What a Graph is, check out [Graph FAQ Section A](graphs_faq.ipynb)
- What parameters we can pass to the `SoccerGraphConverter`, check out [Graph FAQ Section B](graphs_faq.ipynb)
- What features each Graph has, check out [Graph FAQ Section C](graphs_faq.ipynb)

------

To get started we need to load our tracking data using Kloppy, and subsequently pass this to the `KloppyPolarsDataset`. 

This `KloppyPolarsDataset` also takes the following optional parameters:
- ball_carrier_threshold: float = 25.0
- max_player_speed: float = 12.0
- max_ball_speed: float = 28.0
- max_player_acceleration: float = 6.0
- max_ball_acceleration: float = 13.5
- orient_ball_owning: bool = True

🗒️ KloppyPolarsDataset sets the orientation to `Orientation.BALL_OWNING_TEAM` (ball owning team plays left to right) when `orient_ball_owning=True`. This is preferred behaviour in this use-case.

If our dataset does not have the ball owning team we infer the ball owning team automatically using the `ball_carrier_threshold` and subsequently change the orientation automatically to be left to right for the ball owning team too. Additionally, we automatically identify the ball carrying player as the player on the ball owning team closest to the ball.

🗒️ In `SoccerGraphConverter` [deprecated] if the ball owning team was not available we set the orientation to STATIC_HOME_AWAY meaning attacking could happen in two directions. 

<div style="border: 2px solid #ddd; border-radius: 5px; padding: 10px; background-color: ##282C34;">
<pre>
kloppy_dataset = sportec.load_open_tracking_data(
    match_id=match_id,
    coordinates="secondspectrum",
    alive_only=True,
    limit=500,  
)
kloppy_polars_dataset = KloppyPolarsDataset(
    kloppy_dataset=kloppy_dataset,
    ball_carrier_threshold=25.0
)
</pre>
</div>

#### Graph Identifier(s):
After loading the `kloppy_polars_dataset` we now add graph identifiers. We can do this by passing a list of column names on which we want to split our data.

🗒️ When training a model on tracking data it's highly recommended to split data into test/train(/validation) sets by match or period such that all data end up in the same test, train or validation set. This should be done to avoid leaking information between test, train and validation sets. Correctly splitting the final dataset in train, test and validiation sets using these Graph Identifiers is incorporated into `GraphDataset` (see [Section 6.1](#61-split-dataset) for more information).

<div style="border: 2px solid #ddd; border-radius: 5px; padding: 10px; background-color: ##282C34;">
<pre>
kloppy_polars_dataset.add_graph_ids(by=["period_id", "match_id"])
</pre>
</div>

#### Labels:
For training any model we need labels. If your labels are stored in some other dataset format, for example a CSV file, you can join those labels on your `kloppy_polars_dataset.data` (which is a polars DataFrame).

🗒️ For this tutorial we'll use the `dummy_labels()` method that assigns a random binary label to each frame. In a real scenario, you would join your actual labels here.

<div style="border: 2px solid #ddd; border-radius: 5px; padding: 10px; background-color: ##282C34;">
<pre>
kloppy_polars_dataset.add_dummy_labels()
</pre>
</div>

#### Initialize SoccerGraphConverter:
After loading the `kloppy_polars_dataset`, adding graph identifiers, and adding labels, we can initialize the `SoccerGraphConverter`.

The `SoccerGraphConverter` takes many optional parameters to customize how graphs are constructed. For a complete overview, see the [Graph FAQ Section B](graphs_faq.ipynb).

Key parameters include:
- `self_loop_ball`: Whether to add self-loops to the ball node
- `adjacency_matrix_connect_type`: How to connect nodes ("ball", "delaunay", "radius", etc.)
- `adjacency_matrix_type`: Type of adjacency matrix ("split_by_team", "dense", "delaunay")
- `label_type`: Type of label ("binary", "multilabel")
- `defending_team_node_value`: Value to assign to defending team nodes
- `non_potential_receiver_node_value`: Value to assign to non-potential receiver nodes
- `random_seed`: Random seed for reproducibility
- `pad`: Whether to pad graphs to a fixed size
- `verbose`: Whether to print progress information

<div style="border: 2px solid #ddd; border-radius: 5px; padding: 10px; background-color: ##282C34;">
<pre>
converter = SoccerGraphConverter(
    dataset=kloppy_polars_dataset,
    self_loop_ball=True,
    adjacency_matrix_connect_type="ball",
    adjacency_matrix_type="split_by_team",
    label_type="binary",
    defending_team_node_value=0.1,
    non_potential_receiver_node_value=0.1,
    random_seed=42,
    pad=False,
    verbose=False,
)
</pre>
</div>

-----

### 4. Load Kloppy Data, Convert and Store

Now we'll put it all together and process multiple matches from the Sportec dataset.

We'll:
1. Loop through multiple match IDs
2. Load each match with Kloppy
3. Create a KloppyPolarsDataset
4. Add graph identifiers and labels
5. Convert to PyTorch Geometric graphs
6. Store all graphs for training

In [3]:
from unravel.utils import GraphDataset

# Select matches to process
match_ids = ["J03WMX", "J03WN1", "J03WPY"]  # Add more as needed

all_graphs = []

for match_id in match_ids:
    print(f"Processing match: {match_id}")

    # Load Kloppy dataset
    kloppy_dataset = sportec.load_open_tracking_data(
        match_id=match_id,
        only_alive=True,
        limit=500,  # Remove this limit for full match processing
    )

    # Create KloppyPolarsDataset
    kloppy_polars_dataset = KloppyPolarsDataset(
        kloppy_dataset=kloppy_dataset,
        ball_carrier_threshold=25.0,
    )

    # Add graph identifiers
    kloppy_polars_dataset.add_graph_ids()

    # Add labels (in practice, you would join your actual labels here)
    kloppy_polars_dataset.add_dummy_labels()

    # Initialize converter with desired settings
    converter = SoccerGraphConverter(
        dataset=kloppy_polars_dataset,
        self_loop_ball=True,
        adjacency_matrix_connect_type="ball",
        adjacency_matrix_type="split_by_team",
        label_type="binary",
        defending_team_node_value=0.1,
        non_potential_receiver_node_value=0.1,
        random_seed=42,
        pad=False,
        verbose=False,
    )

    # Convert to PyTorch Geometric graphs
    graphs = converter.to_pytorch_graphs()
    all_graphs.extend(graphs)

    print(f"  Generated {len(graphs)} graphs from match {match_id}")

print(f"\nTotal graphs generated: {len(all_graphs)}")

Processing match: J03WMX
  Generated 500 graphs from match J03WMX
Processing match: J03WN1
  Generated 500 graphs from match J03WN1
Processing match: J03WPY
  Generated 500 graphs from match J03WPY

Total graphs generated: 1500


-----

### 5. Creating a Custom Graph Dataset

Now that we have all our graphs, we'll create a `GraphDataset` that makes it easy to work with PyTorch Geometric.

The `GraphDataset` class provides:
- Easy data splitting (train/test/validation)
- Compatibility with PyTorch Geometric DataLoaders
- Automatic batching capabilities

In [4]:
# Create the dataset
dataset = GraphDataset(graphs=all_graphs, format="pyg")

print(f"Dataset contains {len(dataset)} graphs")
print(f"First graph: {dataset[0]}")

Dataset contains 1500 graphs
First graph: Data(x=[23, 15], edge_index=[2, 287], edge_attr=[287, 6], y=[1], id='DFL-MAT-J03WMX-1', frame_id=10534, ball_owning_team_id='DFL-CLU-000008')


-----

### 6. Prepare for Training

Now we'll prepare everything needed to train our Graph Neural Network.

#### 6.1 Split Dataset

We split the dataset into train, test, and validation sets. The `split_test_train_validation` method respects the graph identifiers we set earlier, ensuring that all frames from the same period/match stay together.

🗒️ This prevents data leakage between sets, which is crucial for accurate model evaluation.

In [5]:
# Split the dataset: 60% train, 20% test, 20% validation
train, test, val = dataset.split_test_train_validation(
    split_train=3,
    split_test=1,
    split_validation=1,
    random_seed=42,
)

print(f"Training set: {len(train)} graphs")
print(f"Test set: {len(test)} graphs")
print(f"Validation set: {len(val)} graphs")

Training set: 900 graphs
Test set: 300 graphs
Validation set: 300 graphs


#### 6.2 Model Configurations

We'll set up the model hyperparameters. These settings are based on the research paper mentioned earlier.

In [6]:
# Model hyperparameters
n_layers = 3
channels = 128
drop_out = 0.5
n_out = 1  # Binary classification

# Training hyperparameters
batch_size = 32
max_epochs = 50
learning_rate = 0.001

#### 6.3 Build GNN Model

We initialize the PyTorch Lightning model, which wraps our PyTorch Geometric GNN with training logic, metrics, and optimization.

In [7]:
from unravel.classifiers import PyGLightningCrystalGraphClassifier
import pytorch_lightning as pyl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Initialize the Lightning model
model = PyGLightningCrystalGraphClassifier(
    n_layers=n_layers,
    channels=channels,
    drop_out=drop_out,
    n_out=n_out,
)

#### 6.4 Create DataLoaders

PyTorch Geometric DataLoaders handle batching and shuffling of our graph data.

In [8]:
from torch_geometric.loader import DataLoader

# Create data loaders
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

print(f"Train loader: {len(train_loader)} batches")
print(f"Validation loader: {len(val_loader)} batches")
print(f"Test loader: {len(test_loader)} batches")

Train loader: 29 batches
Validation loader: 10 batches
Test loader: 10 batches


-----

### 7. Training and Prediction

Now we're ready to train our model!

#### 7.1 Initialize Trainer

PyTorch Lightning's Trainer handles the training loop, logging, and callbacks.

In [9]:
# Set up callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="models/",
    filename="best-model-{epoch:02d}-{val_auc:.2f}",
    save_top_k=1,
    monitor="val_auc",
    mode="max",
)

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min",
)

# Initialize trainer
trainer = pyl.Trainer(
    max_epochs=max_epochs,
    accelerator="auto",  # Automatically uses GPU if available
    callbacks=[checkpoint_callback, early_stop_callback],
    log_every_n_steps=10,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


#### 7.2 Train Model

Now we train the model on our data.

In [10]:
# Train the model
trainer.fit(model, train_loader, val_loader)

/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /Users/jbekkers/PycharmProjects/unravelsports/models exists and is not empty.
W1127 14:30:43.342000 97361 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/utilities/model_summary/model_summary.py:493: The total number of parameters detected may be inaccurate because the model contains an instance of `UninitializedParameter`. To get an accurate number, set `self.example_input_array` in your LightningModule.

  | Name      | Type                      | Params | Mode 
----------------------------------------------------------------
0 | model     | PyGCrystalGraphClassifier | 328 K  | train
1 | criterion | BCELoss                   | 0      | train
2 | train_au

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 736. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 92. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

#### 7.3 Save & Load Model

PyTorch Lightning makes it easy to save and load models.

In [11]:
# The best model is already saved by the checkpoint callback
# But we can also manually save
model_path = "models/my-graph-classifier.ckpt"
trainer.save_checkpoint(model_path)

In [12]:
# Load the model
loaded_model = PyGLightningCrystalGraphClassifier.load_from_checkpoint(model_path)

#### 7.4 Evaluate Model

Let's evaluate the model on our test set.

🗒️ Our performance might be poor because we're using random labels, limited epochs, and a small dataset.

📖 For more information on evaluation in sports analytics see: [Methodology and evaluation in sports analytics: challenges, approaches, and lessons learned {J. Davis et. al. (2024)}](https://link.springer.com/article/10.1007/s10994-024-06585-0)

In [13]:
# Test the model
test_results = trainer.test(model, test_loader)

print(test_results)

/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.46666666865348816    │
│         test_auc          │    0.5134821534156799     │
│         test_loss         │    0.6954680681228638     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6954680681228638, 'test_auc': 0.5134821534156799, 'test_acc': 0.46666666865348816}]


#### 7.5 Predict on New Data

1. Load new, unseen data from the Sportec dataset.
2. Convert this data, making sure we use the exact same settings as in step 4.
3. If we set `prediction=True` we do not have to supply labels to the `SoccerGraphConverter`.
4. We do still need to add graph_ids. It is advised to do this by "frame_id" for the prediction, such that we can more easily merge the predictions back to the correct frame

In [14]:
# Load a new match for prediction (one we haven't used for training)
kloppy_dataset = sportec.load_open_tracking_data(
    match_id="J03WR9",  # A game we have not yet used in section 4
    only_alive=True,
    limit=500,
)

pred_kloppy_polars = KloppyPolarsDataset(
    kloppy_dataset=kloppy_dataset,
    ball_carrier_threshold=25.0,
)
pred_kloppy_polars.add_graph_ids(by=["frame_id"])

# Create converter with same settings as training, but with prediction=True
preds_converter = SoccerGraphConverter(
    dataset=pred_kloppy_polars,
    # Settings (MUST match training settings)
    prediction=True,
    self_loop_ball=True,
    adjacency_matrix_connect_type="ball",
    adjacency_matrix_type="split_by_team",
    label_type="binary",
    defending_team_node_value=0.1,
    non_potential_receiver_node_value=0.1,
    random_seed=False,
    pad=False,
    verbose=False,
)

4. Make a prediction on all the frames of this dataset using `trainer.predict`

In [15]:
import torch

# Compute the graphs and add them to the GraphDataset
pred_graphs = preds_converter.to_pytorch_graphs()
pred_dataset = GraphDataset(graphs=pred_graphs, format="pyg")

# Create data loader for predictions
pred_loader = DataLoader(pred_dataset, batch_size=batch_size, shuffle=False)

# Get predictions
predictions = trainer.predict(model, pred_loader)

# Concatenate predictions from all batches
all_predictions = torch.cat(predictions).cpu().numpy()

print(f"Generated {len(all_predictions)} predictions")
print(f"Prediction shape: {all_predictions.shape}")

/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Generated 500 predictions
Prediction shape: (500,)


5. Create a predictions Polars DataFrame

We merge the `preds_df` to the `KloppyPolarsDataset` named `pred_kloppy_polars` that we just applied the predictions to.

🗒️ We use `"frame_id"` here because the prediction dataset "x.id" is the Graph Ids we added above. We did this on the `"frame_id"` too.

In [20]:
import polars as pl

# Create predictions DataFrame
preds_df = pl.DataFrame(
    {"frame_id": [int(x.id) for x in pred_dataset], "y_hat": all_predictions.flatten()}
)
preds_df = preds_df.sort("y_hat")

# Join predictions back to the original data
pred_kloppy_polars.data = pred_kloppy_polars.data.join(
    preds_df, on="frame_id", how="left"
)

# Display a sample of predictions
print("\nSample predictions:")
pred_kloppy_polars.data[295:305][["frame_id", "period_id", "timestamp", "y_hat"]]


Sample predictions:


frame_id,period_id,timestamp,y_hat
i64,i64,duration[μs],f32
10012,1,480ms,0.485382
10012,1,480ms,0.485382
10012,1,480ms,0.485382
10012,1,480ms,0.485382
10013,1,520ms,0.485365
10013,1,520ms,0.485365
10013,1,520ms,0.485365
10013,1,520ms,0.485365
10013,1,520ms,0.485365
